**General Imports**

In [ ]:
from operator import add
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from requests_oauthlib import OAuth1Session
from time import gmtime, strftime

import ast
import json
import nltk
import requests
import requests_oauthlib
import string
import time

**NLTK Imports**

In [ ]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

import nltk

In [ ]:
BATCH_INTERVAL = 5

In [ ]:
ssc = StreamingContext(sc, BATCH_INTERVAL)

### Training the Sentiment Analysis Classifier

An essential part of creating a sentiment analysis algorithm (or any data mining algorithm) is to use a comprehensive dataset or "Corpus" for learning, as well as a set of test data to ensure that the accuracy of the your algorithm meets the standards you expect.

This will also allow you to adjust your algorithm in order to predict better (or more precisely) natural language features that you could extract from the text that will contribute to the sentiment classification, rather than using a generic approach.

We will use the training dataset provided by the University of Michigan for Kaggle competitions: https://inclass.kaggle.com/c/si650winter11

This dataset contains 1,578,627 classified tweets and each row is marked as:

- 1 for positive feeling
- 0 for negative feeling

In [ ]:
file = sc.textFile('aux/datasets/sentiment-analysis.csv')

In [ ]:
header = file.take(1)[0]

In [ ]:
dataset = file.filter(lambda line: line != header)

In [ ]:
dataset.take(3)

In [ ]:
def clearLine(line):
    columns = line.split(',')
    translator = str.maketrans({key: None for key in string.punctuation})
    
    sentiment = columns[1]
    tweet = columns[3].strip()
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    
    tweetLower = [word.lower() for word in tweet]
    
    return(tweetLower, sentiment)

In [ ]:
datasetTraining = dataset.map(lambda line: clearLine(line))

In [ ]:
datasetTraining.take(3)

In [ ]:
nltk.download('stopwords')

In [ ]:
stopWords = [[word, word + '_NEG'] for word in stopwords.words('english') for singleWord in word]
stopWords = [word for wordPair in stopWords for word in wordPair]

In [ ]:
sentimentAnalyzer = SentimentAnalyzer()

In [ ]:
datasetTrainingSample = datasetTraining.take(10000)

In [ ]:
wordsNegative = sentimentAnalyzer.all_words([mark_negation(word) for word in datasetTrainingSample])
wordsNegative = [word for word in wordsNegative if word not in stopWords]

In [ ]:
unigramFeatures = sentimentAnalyzer.unigram_word_feats(wordsNegative, top_n = 200)

In [ ]:
sentimentAnalyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigramFeatures)

In [ ]:
trainingSet = sentimentAnalyzer.apply_features(datasetTrainingSample)

### Model Training

In [ ]:
trainer = NaiveBayesClassifier.train

In [ ]:
classifier = sentimentAnalyzer.train(trainer, trainingSet)

In [ ]:
testSentence01 = [(['this', 'program', 'is', 'bad'], '')]
testSentence02 = [(['tough', 'day', 'at', 'work', 'today'], '')]
testSentence03 = [(['that', 'place', 'is', 'awesome'], '')]

In [ ]:
testSet01 = sentimentAnalyzer.apply_features(testSentence01)
testSet02 = sentimentAnalyzer.apply_features(testSentence02)
testSet03 = sentimentAnalyzer.apply_features(testSentence03)

**Twitter Authentication**

In [ ]:
apiKey = ''
apiKeySecret = ''
accessToken = ''
accessTokenSecret = ''

In [ ]:
searchTerm = 'Trump'

In [ ]:
urlSample = 'https://stream.twitter.com/1.1/statuses/sample.json'

In [ ]:
urlFilter = 'https://stream.twitter.com/1.1/statuses/filter.json?track=' + searchTerm

In [ ]:
auth = requests_oauthlib.OAuth1(apiKey, apiKeySecret, accessToken, accessTokenSecret)

**Stream Configuration**

In [ ]:
rdd = ssc.sparkContext.parallelize([0])

In [ ]:
stream = ssc.queueStream([], default = rdd)

In [ ]:
NUM_TWEETS = 500

In [ ]:
def twitterDataStream():
    response = requests.get(urlFilter, auth = auth, stream = True)
    print(urlFilter, response)
    
    count = 0
    
    for line in response.iter_lines():
        try:
            if count > NUM_TWEETS:
                break

            post = json.loads(line.decode('utf-8'))
            contents = [post['text']]
            count += 1

            yield str(contents)
        except:
            result = False

In [ ]:
def mapData(t, rdd):
    return rdd.flatMap(lambda x: twitterDataStream())

In [ ]:
stream = stream.transform(mapData)

In [ ]:
streamCoord = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
def tweetClassifier(tweet):
    sentence = [(tweet, '')]
    testSet = sentimentAnalyzer.apply_features(sentence)
    
    print(tweet, classifier.classify(testSet[0][0]))
    
    return(tweet, classifier.classify(testSet[0][0]))

In [ ]:
def getTweetText(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        
        tweetLower = [word.lower() for word in tweet]
        
        return tweetClassifier(tweetLower)

In [ ]:
results = []

In [ ]:
def rddOutput(rdd):
    global results
    
    pairs = rdd.map(lambda x: (getTweetText(x)[1], 1))
    counts = pairs.reduceByKey(add)
    output = [count for count in counts.collect()]
    
    result = [time.strftime('%I:%M:%S'), output]
    results.append(result)
    
    print(result)    

In [ ]:
streamCoord.foreachRDD(lambda t, rdd: rddOutput(rdd))

### Start Streaming

In [ ]:
ssc.start()

In [ ]:
count = True

In [ ]:
while count:
    if len(results) > 5:
        count = False

In [ ]:
outputFile = 'aux/' + time.strftime('%I%M%S')

In [ ]:
rddResults = sc.parallelize(results)

In [ ]:
rddResults.saveAsTextFile(outputFile)

In [ ]:
rddResults.collect()

In [ ]:
ssc.stop()